<a href="https://colab.research.google.com/github/olegkorol/backtrader_yahoo/blob/main/backtrader_yahoo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Backtesting trading strategies with Backtrader and Yahoo Finance

## Install and import dependencies

In [ ]:
# Install pip packages in the current Jupyter kernel
# https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/#How-to-use-Pip-from-the-Jupyter-Notebook
import sys
!{sys.executable} -m pip install backtrader pyfolio

In [ ]:
import datetime
import backtrader as bt

## Set desired tickers and start date

In [ ]:
tickers = ['NIO', 'SNE', 'GOOG', 'NFLX', 'DIS',
           'TWTR', 'TSLA', 'AAPL', 'AMZN', 'BTC-USD', 'ETH-USD', 'MSFT', 'KO']
fromdate = datetime.datetime(2018, 1, 1)
todate = datetime.datetime.now()

## Define strategies

In [ ]:
class SmaCrossoverStochastic(bt.Strategy):
    params = dict(sma_fast_period=10, sma_slow_period=30,
                  stochastic_upperband=60, stochastic_lowerband=30)

    def __init__(self):
        sma_fast = bt.indicators.SMA(period=self.p.sma_fast_period)
        sma_slow = bt.indicators.SMA(period=self.p.sma_slow_period)
        self.crossover = bt.indicators.CrossOver(
            sma_fast, sma_slow)  # returns: -1, 0 or 1
        self.stochastic = bt.indicators.StochasticSlow(
            upperband=self.p.stochastic_upperband, lowerband=self.p.stochastic_lowerband)

    def next(self):
        close_price = self.data.close * 1.0
        position_size = self.position.size
        available_cash = self.broker.getcash() * 1

        if self.position.size:
            if self.crossover < 0 and self.stochastic <= self.p.stochastic_lowerband:
                print('- [{}] Closing {} position(s) at {}$'.format(
                    self.data.datetime.date(), position_size, close_price))
                self.close()
        elif self.crossover > 0 and self.stochastic >= self.p.stochastic_upperband:
            print('+ [{}] Buying {} position(s) at {}$'.format(
                self.data.datetime.date(), int(available_cash * 0.95 / close_price), close_price))
            self.buy(size=int(available_cash * 0.95 / close_price))

## Run backtesting

In [ ]:
for ticker in tickers:
    cerebro = bt.Cerebro()
    # The starting capital can be changed here
    cerebro.broker.set_cash(50000)

    data = bt.feeds.YahooFinanceData(
        dataname=ticker,
        fromdate=fromdate,
        todate=todate,
        adjclose=True,
        period='d',
        name='days'
    )
    cerebro.adddata(data)

    # Select Strategy
    cerebro.addstrategy(SmaCrossoverStochastic)

    # Select Analyzer (optional)
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='_sharpe')

    initial_value = cerebro.broker.get_value()
    print('=' * 5, '{}'.format(ticker), '=' * 5)
    print('Start value:', initial_value, '$')

    results = cerebro.run()
    strat = results[0]
    sharpe_ratio = strat.analyzers.getbyname('_sharpe').get_analysis()

    print('End value: %.2f' % cerebro.broker.get_value(), '$')
    print('(Available cash: %.2f' % cerebro.broker.get_cash(), '$)')
    print('*' * 18, '\nCHANGE: {:.4f}%'.format(
        float(cerebro.broker.get_value() / float(initial_value)) * 100.00 - 100))
    print('*' * 18, '\n')
    print('Sharpe Ratio: {}'.format(sharpe_ratio['sharperatio']))

    cerebro.plot(iplot=False)

    print('=' * 5, '{}'.format(ticker), '=' * 5, '\n')